In [11]:
from model.classifier import pointnet_cls
import torch

classifier = pointnet_cls.get_model()

In [12]:
#forward
B = 4
sample_data = torch.randn(B, 6, 1)
unpruned_out = classifier(sample_data)

In [13]:
from torch import nn
from torch.nn.utils import prune

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

module_list = list(classifier.named_modules())
convs = [(module_list[i][1], module_list[i+1][1]) for i in range(len(module_list) - 1) if isinstance(module_list[i][1], nn.Conv1d) and isinstance(module_list[i+1][1], nn.Conv1d)]

for conv_pair in convs:
    print(conv_pair)
    print(conv_pair[0])
    pruned_weight = prune.ln_structured(conv_pair[0], 'weight', 0.5, 2, 1)
    pruned_weight = prune.ln_structured(conv_pair[1], 'weight', 0.5, 2, 0)
    print(pruned_weight)
    break
    prune.ln_structured(conv_pair[1], 'weight', 0.5, 2, 0)

(Conv1d(6, 64, kernel_size=(1,), stride=(1,)), Conv1d(64, 128, kernel_size=(1,), stride=(1,)))
Conv1d(6, 64, kernel_size=(1,), stride=(1,))
Conv1d(64, 128, kernel_size=(1,), stride=(1,))


In [14]:
from torch import nn
from torch.nn.utils import prune

def prune_pointnet(pointnet):
    module_list = list(classifier.named_modules())
    convs = [(module_list[i][1], module_list[i+1][1]) \
            for i in range(len(module_list) - 1) if \
            isinstance(module_list[i][1], nn.Conv1d) \
            and isinstance(module_list[i+1][1], nn.Conv1d)
            ]

    for conv_pair in convs:
        prune.ln_structured(conv_pair[0], 'weight', 0.5, 2, 1)
        prune.ln_structured(conv_pair[1], 'weight', 0.5, 2, 0)

In [15]:
prune_pointnet(classifier)
pruned_out = classifier(sample_data)
torch.sum(torch.square(pruned_out - unpruned_out))

tensor(31.6658, grad_fn=<SumBackward0>)